In [11]:
from typing import Optional
from datetime import timedelta
import pandas as pd
import plotly.express as px

def plot_one_sample(
    example_id: int,
    features: pd.DataFrame,
    df_processed: pd.DataFrame,
    targets: Optional[pd.Series] = None,
    predictions: Optional[pd.Series] = None,
    display_title: Optional[bool] = True,
):
    # Obtener las features para ese ejemplo concreto
    features_ = features.loc[example_id]
    
    # Obtener el target (valor real)
    target_ = targets.loc[example_id] if targets is not None else None
    
    # Extraer columnas que son lags
    ts_columns = [c for c in features.columns if c.startswith('rides_previous_')]
    
    # Valores históricos + el target
    ts_values = [features_[c] for c in ts_columns] + [target_]
    
    # Crear el rango de fechas (lags + target)
    ts_dates = pd.date_range(
        df_processed.loc[example_id, 'pickup_hour'] - timedelta(hours=len(ts_columns)),
        df_processed.loc[example_id, 'pickup_hour'],
        freq='H'
    )

    # Crear gráfico interactivo con Plotly
    title = f'Pick up hour={df_processed.loc[example_id, "pickup_hour"]}, location_id={df_processed.loc[example_id, "pickup_location_id"]}' if display_title else None
    fig = px.line(
        x=ts_dates, y=ts_values,
        template='plotly_dark',
        markers=True, title=title
    )
    
    # Añadir target real como punto verde
    if targets is not None:
        fig.add_scatter(x=ts_dates[-1:], y=[target_],
                        line_color='green',
                        mode='markers', marker_size=10, name='actual value') 
        
    # Añadir predicción como X roja si existe
    if predictions is not None:
        prediction_ = predictions.iloc[example_id]
        fig.add_scatter(x=ts_dates[-1:], y=[prediction_],
                        line_color='red',
                        mode='markers', marker_symbol='x', marker_size=15,
                        name='prediction')             
    
    fig.show()

# ------------ USO ------------

df_processed = pd.read_parquet('../data/processed/df_lags.parquet')
df_features = pd.read_parquet('../data/processed/X.parquet')
df_target = pd.read_parquet('../data/processed/y.parquet')

# Ahora simplemente ejecuta:
plot_one_sample(
    example_id=43, 
    features=df_features, 
    df_processed=df_processed,
    targets=df_target['rides']
)
